In [ ]:
!pip install -U sentence-transformers

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import pandas as pd
from sentence_transformers import SentenceTransformer
import pickle

class Similarity:
  def __init__(self):

    self.model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose a different model based on your needs
    self.Data_Empty = pd.DataFrame([])#columns= ["ID","Q0","Doc_Name","Score"])
  def load_dataset(self, dataset_path):
    self.dataset = np.load(dataset_path)
    print(self.dataset.shape)

  def find_similar_documents(self,top_k):

    #print(f"\nIn Find Similarmethod {self.query} is executing...\n")
    query_embedding = self.model.encode(self.query,show_progress_bar = True,convert_to_tensor= True)

    #print(query_embedding, self.dataset[:,:-1].shape)
    query_embedding = query_embedding.reshape(1, -1)
    #print(f"\n Shapeof Query Embedding{query_embedding.shape}")
    self.cosine_scores = cosine_similarity(query_embedding, self.dataset[:,:-1]).flatten()

    self.top_results = np.argsort(self.cosine_scores)[-top_k:][::-1]
    #self.cosine_scores = self.cosine_scores.sort()[-top_k:][::-1]
    self.cosine_scores = self.cosine_scores[self.top_results]
    #print(f"\n\nTop results \t:\t{self.top_results}")
    #print(f"\n\nMaximum index as top results \t:\t{self.top_results[0].max()}")
    return self.top_results[:top_k],self.cosine_scores[:top_k]
  def process(self,dataset_path:str,query,top_k:int,filename:str):
    self.load_dataset(dataset_path)
    self.LabelDecoder()
    q_text = query['Text'].tolist()
    q_Id   = query["ID"].tolist()
    #q_text = query["Text
    #print(q_text)

    for q in tqdm(range(len(q_text))):
      self.query = q_text[q]
      tmp_q = [q_Id[q][3:]] * top_k   # Query ID is en_140 ID should be 140
      #print(tmp_q)
      tmp_q0 = ['Q0']*top_k
      self.top,self.score = self.find_similar_documents(top_k)
      #print(type(self.top))
      self.top = [self.dataset[i,-1] for i in self.top]
      self.top = [self.label[int(i)] for i in self.top]
      Rank =  range(1,top_k+1)
      self.score = np.round(self.score,2)
      #self.top = pd.DataFrame(self.top,columns = ['Doc_Name'])
      #self.score = pd.DataFrame(self.score,columns = ['Score'])
      self.DataQ_Rel_Frame = pd.DataFrame([tmp_q,tmp_q0,self.top,Rank,self.score]).T
      self.Data_Empty = pd.concat([self.DataQ_Rel_Frame,self.Data_Empty],axis = 0,ignore_index=True)
      print(self.Data_Empty.shape)
    self.Data_Empty.columns = ['ID','Q0','Doc_Name','Rank','Score']
    self.Data_Empty.to_csv(PATH+filename)
  # def _process(self, QID,DOCNAME,SCORE):

  #   return pd.DataFrame([QID,DOCNAME,SCORE])

  def LabelDecoder(self):
    self.label = pickle.load(open(PATH+'VectorDB_1000/LabelDecoder1.pkl','rb'))
    return self.label




ModuleNotFoundError: No module named 'sentence_transformers'

# Train Q-Relative

In [2]:
PATH            = "/content/drive/MyDrive/SQCLIR_IITM/Final_en_corpus/"
dataset_path    = PATH+'VectorDB_1000/Dataset_447543_385.npy'
Query_similar   = Similarity()
#Query_similar.load_dataset(dataset_path)
#top,score       = Query_similar.find_similar_documents(top_k = 1000)
Query_similar.process(dataset_path,en_wave_query_train,999,filename = "Predicted_Q_rel_Train_14_10.csv")


NameError: name 'Similarity' is not defined

#Test query

In [ ]:
PATH            = "/content/drive/MyDrive/SQCLIR_IITM/Final_en_corpus/"
dataset_path    = PATH+'VectorDB_1000/Dataset_447543_385.npy'
Query_similar   = Similarity()
#Query_similar.load_dataset(dataset_path)
#top,score       = Query_similar.find_similar_documents(top_k = 1000)
Query_similar.process(dataset_path,en_wave_query_test,1000,filename = "Predicted_Q_rel_Test_14_10_1000.csv")
import pandas as pd
Test = pd.read_csv(PATH+"Predicted_Q_rel_Test_14_10_1000.csv")
Test.head()
Test['Run_ID'] =['en-IITM_BS-AEKD-run1']*Test.shape[0]
Test.head()
Test.drop(['Unnamed: 0'],axis = 1,inplace = True)
# Step 2: Write the DataFrame to a TXT file
txt_file_path = PATH+'en-IITM_MiniLM_1000.txt'  # Replace with your desired TXT file path
Test.to_csv(txt_file_path, sep='\t', index=False, header=True)  # Use tab as a separator
#
Test.head()